# События виджетов

В этой лекции мы обсудим события виджетов, такие как нажатие кнопок!

## Специальные события

`Button` не используется для представления типа данных. Вместо этого, виджет button используется для работы с нажатиями мыши. Метод `on_click` виджета `Button` можно использовать для регистрации функции, которая будет запускаться при нажатии кнопки. Описание docstring для `on_click` можно посмотреть ниже.

In [ ]:
import ipywidgets as widgets

print(widgets.Button.on_click.__doc__)

### Пример #1 - on_click

Поскольку нажатия кнопок это информация stateless, они передаются из фронт-энда в бэк-энд с помощью специальных сообщений. С помощью метода `on_click`, кнопка button отображает сообщение при её нажатии, как показано ниже.

In [ ]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)

### Пример #2 - on_submit

Виджет `Text` также имеет специальное событие `on_submit`. Событие `on_submit` происходит, когда пользователь нажимает клавишу <kbd>enter</kbd>.

In [ ]:
text = widgets.Text()
display(text)

def handle_submit(sender):
    print(text.value)

text.on_submit(handle_submit)

## События Traitlet
Свойства виджетов являются IPython traitlets, и traitlet'ы могут содержать события. Чтобы реагировать на изменения, можно использовать метод `observe` для регистрации обратного вызова. Описание docstring для `observe` приведено ниже.

In [ ]:
print(widgets.Widget.observe.__doc__)

### Signatures
Как указано в docstring, регистрируемый обратный вызов должен иметь вид `handler(change)`, где `change` это словарь, содержащий информацию об изменении.

Ниже приведён пример использования этого метода, в котором отображается значение `IntSlider` при его изменении.


In [ ]:
int_range = widgets.IntSlider()
display(int_range)

def on_value_change(change):
    print(change['new'])

int_range.observe(on_value_change, names='value')

# Связь между виджетами
Зачастую Вам может понадобиться связать атрибуты виджетов вместе. Синхронизацию атрибутов можно сделать проще, чем просто использование событий traitlets.

## Связь атрибутов traitlets в ядре kernel¶

Первый способ это использование функций `link` и `dlink` из модуля `traitlets`. Это работает, только если Вы взаимодействуете с работающим ядром kernel.


In [ ]:
import traitlets

In [ ]:
# Create Caption
caption = widgets.Label(value = 'The values of slider1 and slider2 are synchronized')

# Create IntSliders
slider1 = widgets.IntSlider(description='Slider 1')
slider2 =  widgets.IntSlider(description='Slider 2')

# Use trailets to link
l = traitlets.link((slider1, 'value'), (slider2, 'value'))

# Display!
display(caption, slider1, slider2)

In [ ]:
# Create Caption
caption = widgets.Label(value='Changes in source values are reflected in target1')

# Create Sliders
source = widgets.IntSlider(description='Source')
target1 = widgets.IntSlider(description='Target 1')

# Use dlink
dl = traitlets.dlink((source, 'value'), (target1, 'value'))
display(caption, source, target1)

Функции `traitlets.link` и `traitlets.dlink` возвращают объект `Link` или `DLink`. Связь можно убрать с помощью метода `unlink`.

In [ ]:
# Можно получить ошибку в зависимости от порядка выполнения ячеек!
l.unlink()
dl.unlink()

### Регистрация обратных вызовов для отслеживания изменений в ядре kernel

Поскольку атрибуты виджетов на стороне Python являются traitlet'ами, Вы можете регистрировать обработчики для событий изменения, когда модель получает изменения из фронт-энда.

Обработчик (handler), который передаётся для наблюдения за событием, будет вызван с одним параметром change. Объект change содержит по крайней мере ключ `type` и ключ `name`, которые соответствуют типу нотификации и имени атрибута, который был причиной нотификации.

Также, в зависимости от значения `type`, могут передаваться дополнительные ключи. В случае если тип равен `change`, у нас ещё будут следующие ключи:
* `owner` : экземпляр HasTraits
* `old` : старое значение изменяемого атрибута trait 
* `new` : новое значение изменяемого атрибута trait
* `name` : название изменяемого атрибута trait.


In [ ]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

## Связь атрибутов виджетов со стороны клиента

При синхронизации атрибутов traitlet'ов, Вы можете столкнуться с задержкой по времени, связанной с временем доступа к серверной части. Вы также можете связать атрибуты виджетов напрямую в браузере, используя виджеты link, либо в одном направлении, либо в двух направлениях.

Ссылки Javascript остаются неизменными при помещении виджетов внутри веб-страниц html без ядра kernel.

In [ ]:
# NO LAG VERSION
caption = widgets.Label(value = 'The values of range1 and range2 are synchronized')

range1 = widgets.IntSlider(description='Range 1')
range2 = widgets.IntSlider(description='Range 2')

l = widgets.jslink((range1, 'value'), (range2, 'value'))
display(caption, range1, range2)

In [ ]:
# NO LAG VERSION
caption = widgets.Label(value = 'Changes in source_range values are reflected in target_range')

source_range = widgets.IntSlider(description='Source range')
target_range = widgets.IntSlider(description='Target range')

dl = widgets.jsdlink((source_range, 'value'), (target_range, 'value'))
display(caption, source_range, target_range)

Функция `widgets.jslink` возвращает виджет `Link`. Эта связь может быть удалена с помощью метода `unlink`.

In [ ]:
l.unlink()
dl.unlink()

### Различие между связыванием в ядре kernel и связыванием на клиенте

Связывание в ядре kernel означает связывание с помощью python. Если два слайдера связаны в ядре kernel, то при изменении одного слайдера браузер отправляет сообщение ядру kernel (в данном случае python) с обновлением слайдера, и затем виджет link widget в ядре kernel передаёт это изменение в другой слайдер в ядре kernel, и далее объект другого слайдера отправляет сообщение браузеру обновить представление этого другого слайдера в браузере. Если ядро kernel не выполняется (например, на статичной web-странице), то элементы не будут связаны.

Связывание с помощью jslink (то есть, на стороне браузера) означает создание ссылки в Javascript. Когда один слайдер меняется, работающий в браузере Javascript меняет значение другого слайдера в браузере, без необходимости общаться с ядром kernel. Если слайдеры связаны с объектами kernel, то каждый из слайдеров обновить свои объекты на стороне ядра kernel независимо друг от друга.

Чтобы увидеть разницу между двумя этими типами связывания, можете посмотреть [документацию ipywidgets](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html) и попробовать слайдеры рядом с нижней частью страницы. Те слайдеры, которые связаны в ядре kernel с помощью `link` и `dlink` больше не связаны, а те, что связаны в браузере с помощью `jslink` и `jsdlink`, всё еще связаны.


## Постоянные обновления

Некоторые виджеты с помощью атрибута `continuous_update` предоставляют возможность постоянно обновлять значения, либо обновлять значения только когда пользователь отправляет значение (например, нажимая Enter или переходя на другой элемент). В примере ниже, элементы “Delayed” передают свои значения только после того, как пользователь завершил перемещение слайдера или ввод текста в текстовом поле. Элементы “Continuous” постоянно передают свои значения при их изменении. Чтобы увидеть разницу, попробуйте ввести число из двух цифр в каждом из текстовых полей, или переместить каждый из слайдеров.


In [ ]:
import traitlets
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

traitlets.link((a, 'value'), (b, 'value'))
traitlets.link((a, 'value'), (c, 'value'))
traitlets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

Элементы Слайдер, `Text` и `Textarea` по умолчанию имеют значение `continuous_update=True`. Элементы `IntText` и другие текстовые поля для ввода целых и дробных чисел имеют значение `continuous_update=False` (поскольку в большинстве случаев Вы вводите всё число перед тем, как нажать enter или перейти на другой элемент).

# Резюме
Теперь Вы знаете, как работать с событиями виджетов!